In [1]:
import pandas as pd
from dash import dcc, html, Dash
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
spacex_df.columns.tolist()

['Unnamed: 0',
 'Flight Number',
 'Date',
 'Time (UTC)',
 'Booster Version',
 'Launch Site',
 'Payload',
 'Payload Mass (kg)',
 'Orbit',
 'Customer',
 'Landing Outcome',
 'class',
 'Lat',
 'Long']

In [4]:
# Create a dash application
app = Dash(__name__)

In [5]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}
                                       ),
                                
                                dcc.Dropdown(id='site-dropdown',options=[{'label': 'All Sites', 'value': 'ALL'},
                                                                         {'label': 'Cape Canaveral Launch Complex 40 (CAFS LC-40)', 'value': 'CCAFS LC-40'},
                                                                         {'label': 'Cape Canaveral Space Launch Complex 40 (CCAFS SLC-40)', 'value': 'CCAFS SLC-40'},
                                                                         {'label': 'Kennedy Space Center Launch Complex 39A (KSC LC-39A)', 'value': 'KSC LC-39A'},
                                                                         {'label': 'Vandenberg Air Force Base Space Launch Complex (VAFB SLC-4E)', 'value': 'VAFB SLC-4E'}
                                                                        ],
                                             value='ALL',
                                             placeholder="Select a Launch Site",
                                             searchable=True
                                            ),
                                
                                html.Br(),
                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                
                                html.Br(),
                                
                                html.P("Payload range (Kg):"),
                                
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                value=[min_payload, max_payload]),
                                
                                html.Br(),
                                
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                               ]
                     )

In [6]:
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df.groupby(['Launch Site'], as_index=False).mean()
    if entered_site == 'ALL':
        return px.pie(filtered_df, values='class', names='Launch Site', title='Launch Success Rate For All Sites')
    # return the outcomes in pie chart for a selected site
    filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
    filtered_df['outcome'] = filtered_df['class'].apply(lambda x: 'Success' if (x == 1) else 'Failure')
    filtered_df['counts'] = 1
    return px.pie(filtered_df, values='counts', names='outcome', title='Launch Success Rate For ' + entered_site)

In [7]:
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])

def get_scatter_chart(entered_site, slider):
    filtered_df = spacex_df[
        (slider[0] <= spacex_df['Payload Mass (kg)']) & (spacex_df['Payload Mass (kg)'] <= slider[1])
    ]
    if entered_site == 'ALL':
        return px.scatter(filtered_df,
                          x='Payload Mass (kg)', y='class',
                          color='Booster Version Category',
                          title='Launch Success Rate For All Sites')
    # return the outcomes in pie chart for a selected site
    filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
    filtered_df['outcome'] = filtered_df['class'].apply(lambda x: 'Success' if (x == 1) else 'Failure')
    filtered_df['counts'] = 1
    return px.scatter (filtered_df,
                       x='Payload Mass (kg)', y='class',
                       color='Booster Version Category',
                       title='Launch Success Rate For ' + entered_site)

In [ ]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [12/May/2023 16:12:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2023 16:12:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2023 16:12:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2023 16:12:17] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [12/May/2023 16:12:17] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [12/May/2023 16:12:17] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [12/May/2023 16:12:17] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
[2023-05-12 16:12:17,421] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\DELL\AppData\Roa

127.0.0.1 - - [12/May/2023 16:15:46] "POST /_dash-update-component HTTP/1.1" 500 -
[2023-05-12 16:15:52,118] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py", line 1283, in dispatch
    ctx.run(
  File "C:\ProgramData\Anaconda3\lib\site-pac

127.0.0.1 - - [12/May/2023 16:18:41] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [12/May/2023 16:19:44] "POST /_dash-update-component HTTP/1.1" 200 -
[2023-05-12 16:19:44,762] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py"

127.0.0.1 - - [12/May/2023 16:20:20] "POST /_dash-update-component HTTP/1.1" 500 -
C:\Users\DELL\AppData\Local\Temp\ipykernel_16068\2431199016.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\DELL\AppData\Local\Temp\ipykernel_16068\2431199016.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

127.0.0.1 - - [12/May/2023 16:20:27] "POST /_dash-update-component HTTP/1.1" 200 -
[2023-05-12 16:20:27,264] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
 

127.0.0.1 - - [12/May/2023 16:20:39] "POST /_dash-update-component HTTP/1.1" 500 -
C:\Users\DELL\AppData\Local\Temp\ipykernel_16068\2431199016.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\DELL\AppData\Local\Temp\ipykernel_16068\2431199016.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

127.0.0.1 - - [12/May/2023 16:20:52] "POST /_dash-update-component HTTP/1.1" 200 -
[2023-05-12 16:20:52,645] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
 

127.0.0.1 - - [12/May/2023 16:26:05] "POST /_dash-update-component HTTP/1.1" 500 -
